In [1]:
import pandas as pd
from langchain_community.document_loaders import DirectoryLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import CSVLoader
from langchain_core.documents import Document
import os
import torch
import numpy as np
from pathlib import Path
import datetime
from transformers import AutoModelForSequenceClassification, AutoTokenizer

## 筛选跟每个service program 相关的posts

### Raw Data Match

In [ ]:
# def sentiment_evaluation(directory, output_directory):
#     # 创建输出目录（如果不存在）
#     Path(output_directory).mkdir(parents=True, exist_ok=True)
    
#     # 选择设备
#     if torch.cuda.is_available():
#         device = "cuda:1"
#     elif torch.backends.mps.is_available():
#         device = "mps"
#     else:
#         device = "cpu"
    
#     print(f"设备: {device}")
    
#     # 加载多语言情感分析模型
#     model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual"
#     print(f"\n > 正在加载模型 '{model_name}' ")
    
#     hf_model = AutoModelForSequenceClassification.from_pretrained(model_name)
#     hf_model.to(device)
#     hf_tokenizer = AutoTokenizer.from_pretrained(model_name)
    
#     # 为深圳地铁站定义关键词过滤
#     sz_metro_stations = [
#         "机场东", "机场", "宝安中心", "前海湾", "鲤鱼门", "大新", "桃园", "深大", "科苑", "白石洲", 
#         "世界之窗", "华侨城", "侨城北", "香蜜湖", "车公庙", "竹子林", "招商银行大厦", "深康", "黄贝岭", 
#         "黄贝", "新秀", "莲塘", "梧桐山南", "梧桐山", "盐田路", "沙头角", "海山", "罗湖", "国贸", 
#         "老街", "大剧院", "科学馆", "华强路", "岗厦", "会展中心", "香蜜", "深南香蜜", "红树湾", "后海", 
#         "南山", "科技园", "大学城", "桃源村", "龙珠", "龙华", "清湖", "碧海湾", "铁路公园", "西丽湖"
#     ]
    
#     # 定义关键词用于过滤
#     keywords = ["深圳", "深铁"] + sz_metro_stations
    
#     # 处理目录中的所有CSV文件
#     for csv_file in Path(directory).glob('*.csv'):
#         print(f"\n正在处理文件: {csv_file} 时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
#         # 使用pandas读取CSV
#         df = pd.read_csv(csv_file)
#         print(f"{csv_file.name}中的总行数: {len(df)}")
        
#         # 检查"微博正文"列是否存在
#         if "微博正文" not in df.columns:
#             print(f"警告: '{csv_file}'中未找到'微博正文'列，跳过该文件。")
#             continue
        
#         # 过滤包含深圳地铁相关关键词的行
#         filtered_df = df[df["微博正文"].apply(lambda text: 
#             any(keyword in str(text) for keyword in keywords) if pd.notna(text) else False
#         )]
        
#         print(f"过滤后的行数: {len(filtered_df)}")
        
#         store_posts = []
        
#         # 对每行进行情感分析
#         for index, row in filtered_df.iterrows():
#             if index % 100 == 0:
#                 print(f"已处理 {index} 行，时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
                
#             each_post = row['微博正文']
            
#             # 跳过过长的文本
#             if len(str(each_post)) > 512:
#                 continue
                
#             # 使用模型进行情感分析
#             inputs = hf_tokenizer(str(each_post), return_tensors="pt", truncation=True, max_length=512).to(device)
            
#             with torch.no_grad():
#                 logits = hf_model(**inputs).logits
            
#             logits = logits.cpu().numpy()[0]
            
#             # 使用sigmoid函数归一化
#             normalized = 1 / (1 + np.exp(-logits))
            
#             # 只保存明显的积极或消极情感
#             if not (normalized[0] == max(normalized) or normalized[2] == max(normalized) and abs(max(normalized) - normalized[1]) > 0.1):
#                 continue
                
#             # 将行转换为字典并添加情感分数
#             post_dict = row.to_dict()
#             post_dict['Negative'] = float(normalized[0])
#             post_dict['Neutral'] = float(normalized[1])
#             post_dict['Positive'] = float(normalized[2])
#             store_posts.append(post_dict)
        
#         # 将结果保存为CSV
#         if store_posts:
#             output_df = pd.DataFrame(store_posts)
#             output_filename = f"senti_{csv_file.name}"
#             output_path = os.path.join(output_directory, output_filename)
#             output_df.to_csv(output_path, index=False, encoding='utf-8-sig')
#             print(f"已保存情感分析结果到: {output_path}")
#         else:
#             print(f"警告: 文件 {csv_file.name} 没有符合条件的内容")


# # 设置路径
# directory_path = '深圳地铁'  # 输入目录
# output_directory = '深圳地铁_情感分析'  # 输出目录

# start_time = datetime.datetime.now()
# print(f"情感分析开始时间: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

# sentiment_evaluation(directory_path, output_directory)

# end_time = datetime.datetime.now()
# print(f"情感分析结束时间: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
# time_used = (end_time - start_time).total_seconds() / 60
# print(f"情感分析用时: {time_used:.2f} 分钟")

### 从已经过滤后的数据匹配

In [ ]:
# Directory path
directory_path = 'senti_results_SZ'
all_docs = []

# Define the Shenzhen metro station names
sz_metro_stations = [
    "机场东", "机场", "宝安中心", "前海湾", "鲤鱼门", "大新", "桃园", "深大", "科苑", "白石洲", 
    "世界之窗", "华侨城", "侨城北", "香蜜湖", "车公庙", "竹子林", "招商银行大厦", "深康", "黄贝岭", 
    "黄贝", "新秀", "莲塘", "梧桐山南", "梧桐山", "盐田路", "沙头角", "海山", "罗湖", "国贸", 
    "老街", "大剧院", "科学馆", "华强路", "岗厦", "会展中心", "香蜜", "深南香蜜", "红树湾", "后海", 
    "南山", "科技园", "大学城", "桃源村", "龙珠", "龙华", "清湖", "碧海湾", "铁路公园", "西丽湖"
]

# Define keywords for filtering
keywords = ["深圳", "深铁"] + sz_metro_stations

# Iterate through CSV files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        
        # Use pandas to read the CSV and preserve row information
        df = pd.read_csv(file_path)
        
        # Check if "微博正文" column exists
        if "微博正文" not in df.columns:
            print(f"Warning: '微博正文' column not found in {file_path}. Skipping file.")
            continue
        
        # Filter rows where "微博正文" contains any of the keywords
        filtered_df = df[df["微博正文"].apply(lambda text: 
            any(keyword in str(text) for keyword in keywords) if pd.notna(text) else False
        )]
        
        # Convert each filtered row to a Document with appropriate metadata
        for index, row in filtered_df.iterrows():
            # Convert row to string representation
            content = row["微博正文"]
            # Create metadata with file source and row index
            metadata = {
                'source': file_path,
                'row': index,  # Keep original index from source file
                # You can add other metadata from your CSV if needed
            }
            doc = Document(page_content=content, metadata=metadata)
            all_docs.append(doc)

print(f"Loaded {len(all_docs)} documents (rows) from CSVs after filtering for Shenzhen metro related content")

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
texts_all = text_splitter.split_documents(all_docs)

In [2]:
hg_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

#### Create VectorDB

In [ ]:
# persist_directory = 'docs/chroma_rag/'
# posts_langchain_chroma = Chroma.from_documents(
#     documents=texts_all,
#     collection_name="SZ_posts_senti",
#     embedding=hg_embeddings,
#     persist_directory=persist_directory,
#     collection_metadata={"hnsw:space": "cosine"}
# )

#### Load VectorDB

In [3]:
persist_directory = 'docs/chroma_rag/'
posts_langchain_chroma = Chroma(
    collection_name="SZ_posts_senti",
    persist_directory=persist_directory,
    embedding_function=hg_embeddings,
    collection_metadata={"hnsw:space": "cosine"} # this make sure the similarity socre between [0,1]
)

/var/folders/3z/wdtcvnz52_1ffx0skvy0cyqm0000gn/T/ipykernel_30642/1355148894.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  posts_langchain_chroma = Chroma(


#### Description Data for Service Program

In [ ]:
pd.read_csv("service_program_data/SPD_SZ_zh.csv").head()

In [ ]:
df = pd.read_csv('senti_results_SZ/senti_cleaned_深圳 地铁 201901 1.0.csv')
df.head(5)

In [4]:
import pandas as pd
from IPython.display import display, HTML
import os
import glob

# Create directory for output if it doesn't exist
similarity_threshold = 0.45

output_dir = f'similarity_threshold={similarity_threshold}'
os.makedirs(output_dir, exist_ok=True)

# Load the service program CSV file
service_program_df = pd.read_csv('service_program_data/SPD_SZ_zh.csv')

# Create a cache for the source dataframes to avoid reloading them
source_df_cache = {}

# Iterate through each row of the CSV
for index, row in service_program_df.iterrows():
    # Concatenate the first three columns as service_dimension
    service_dimension = ' '.join([str(row[col]) for col in service_program_df.columns[1:3] 
                            if pd.notna(row[col]) and str(row[col]) != 'nan'])
    
    # Search for similar documents in the vector database
    docs = posts_langchain_chroma.similarity_search_with_relevance_scores(service_dimension, k=10000, score_threshold=similarity_threshold)
    # Create a dictionary to group matched rows by source file
    matches_by_source = {}
    
    # Process each matched document
    # each doc is a tuple
    for doc in docs:
        source = doc[0].metadata.get('source', None)
        row_index = doc[0].metadata.get('row', None)
        
        if source and row_index is not None:
            # Convert row to integer if it's a numeric string
            try:
                row_index = int(row_index)
            except (ValueError, TypeError):
                continue
                
            if source not in matches_by_source:
                matches_by_source[source] = []
            
            matches_by_source[source].append(row_index)
    
    # Create a list to store all matching rows from original sources
    all_matched_rows = []
    
    # For each source file, get the matching rows
    for source, row_indices in matches_by_source.items():
        # Load the source file if not in cache
        if source not in source_df_cache:
            try:
                source_df_cache[source] = pd.read_csv(source)
            except Exception as e:
                print(f"Error loading {source}: {e}")
                continue
        
        source_df = source_df_cache[source]
        
        # Get rows from original source file
        for row_idx in row_indices:
            try:
                if 0 <= row_idx < len(source_df):
                    row_data = source_df.iloc[row_idx].to_dict()
                    row_data['original_source'] = source
                    row_data['original_row'] = row_idx
                    all_matched_rows.append(row_data)
            except Exception as e:
                print(f"Error accessing row {row_idx} in {source}: {e}")
    
    # Convert to DataFrame
    if all_matched_rows:
        matches_df = pd.DataFrame(all_matched_rows)
        
        # Create a filename for this service program
        safe_filename = f"service_program_{index}_matches.csv"
        
        # Save to CSV
        matches_df.to_csv(os.path.join(output_dir, safe_filename), index=False)
    
    # Print progress
    if index % 10 == 0:
        print(f"Processed {index} rows")

print(f"Completed! All match files saved to {output_dir} directory.")

# 创建摘要文件
summary_file_path = os.path.join(output_dir, '0.matched_summary.txt')
with open(summary_file_path, 'w', encoding='utf-8') as summary_file:
    summary_file.write(f"Created CSV files with matching posts:\n")
    for i, filename in enumerate(sorted(os.listdir(output_dir))):
        if filename == '0.matched_summary.txt':  # 跳过摘要文件本身
            continue
        try:
            match_df = pd.read_csv(os.path.join(output_dir, filename))
            summary_line = f"- {filename}: {len(match_df)} matched original posts\n"
            summary_file.write(summary_line)
            print(summary_line, end='')
        except:
            summary_line = f"- {filename}: Could not read file\n"
            summary_file.write(summary_line)
            print(summary_line, end='')

print(f"\nSummary saved to {summary_file_path}")

Processed 0 rows
Processed 10 rows
Processed 20 rows
Completed! All match files saved to similarity_threshold=0.4 directory.
- service_program_0_matches.csv: 965 matched original posts
- service_program_10_matches.csv: 10000 matched original posts
- service_program_11_matches.csv: 10000 matched original posts
- service_program_12_matches.csv: 1089 matched original posts
- service_program_13_matches.csv: 10000 matched original posts
- service_program_14_matches.csv: 4153 matched original posts
- service_program_15_matches.csv: 995 matched original posts
- service_program_16_matches.csv: 10000 matched original posts
- service_program_17_matches.csv: 248 matched original posts
- service_program_18_matches.csv: 10000 matched original posts
- service_program_19_matches.csv: 1218 matched original posts
- service_program_1_matches.csv: 87 matched original posts
- service_program_20_matches.csv: 2547 matched original posts
- service_program_21_matches.csv: 10000 matched original posts
- servic